# Azure AI Agent Service - AI Search

### *Create Azure AI Search agent to search over the documents indexed using Azure AI search service*

<span style="color:yellow"> Please refer *pre-requisite* section in readme file before running this notebook! </span>


#### Step 1

* Rename *credentials.txt* file to *credentials.env* file 

* Add all required configuration values in the *credentials.env* file

* Select kernel as *Python 3* in the top right corner of the notebook


#### Step 2

* Login to Azure account, by following the below steps:

    * Right click on this notebook from left menu and open in terminal
    
    * Run the command `az login` and follow the instructions to login to your Azure account

#### Step 3
* Install the required packages by running the below command

In [ ]:
#install dependencies
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
import os
from datetime import datetime as pydatetime
from typing import Any, List, Dict
from dotenv import load_dotenv
# Azure AI Projects
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import ConnectionType
from azure.ai.projects.models import AzureAISearchTool, AzureAISearchQueryType


Load variables from env file

In [34]:
# load secrets
load_dotenv("credentials.env", override=True)
aiConnectionStr=os.environ["AIPROJECT_CONNECTION_STRING"]
modelName=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
aiSearchConnectionName = os.getenv("AISEARCH_CONNECTION_NAME")
indexName = os.getenv("AISEARCH_INDEX_NAME")

Check version of AI projects library. Make sure you are using version 1.0.0b10

In [22]:
import azure.ai.projects
print(azure.ai.projects.__version__)

1.0.0b10


In [ ]:
# Create Client and Load Azure AI Foundry project client
credential = DefaultAzureCredential(exclude_interactive_browser_credential=False)
project_client = AIProjectClient.from_connection_string(
    credential=credential,
    scope=['https://management.azure.com','https://management.core.windows.net','https://management.core.windows.net/','https://management.core.windows.net','https://management.azure.com/','https://management.azure.com'],
    conn_str=aiConnectionStr,   
)

In [24]:
# List all agents to see what's already available
all_agents = project_client.agents.list_agents().data
print(f"Found {len(all_agents)} agents:")
for agent in all_agents:
	print(f"- {agent.name} (ID: {agent.id})")

Found 7 agents:
- FabricLH2-DataAgent2 (ID: asst_JYvI8L1xuap9zqE5NLVVyvcC)
- Agent193 (ID: asst_KDVZmGnuwyUPAIB9CIHCleRy)
- Agent537 (ID: asst_dGh6zkzAf48NKw1Xyr4JkFtY)
- AISearchAgent (ID: asst_b3Fe2GACrZrmgtJbGHOl6EiM)
- AISearch_Agent (ID: asst_Y90HCxqazgc0LC9MRkDFWURu)
- Agent766 (ID: asst_TLIGQPj8JcsoQpGwJh5wjFTi)
- FabricLH2-DataAgent (ID: asst_yMSAqwYv2pCZs8xuSaNiAPET)


In [ ]:
#AI search connection
aisearch_connection = project_client.connections.get(
    connection_name=aiSearchConnectionName,    
    include_credentials=True,  # Optional. Defaults to "False".
)
print(aisearch_connection)

{
 "name": "aiworkshop2aisearch",
 "id": "/subscriptions/74a008ce-fe34-4b7a-a4e7-012f6ca145c8/resourceGroups/aiworkshop2/providers/Microsoft.MachineLearningServices/workspaces/aiworkshop2-project1/connections/aiworkshop2aisearch",
 "authentication_type": "AAD",
 "connection_type": "ConnectionType.AZURE_AI_SEARCH",
 "endpoint_url": "https://aiworkshop2-aisearch.search.windows.net",
 "key": null
 "token_credential": "REDACTED"
}



------

ex-1: Enter input prompt to search over the documents indexed using Azure AI search service 
and name the agent as *AISearchWorkshopAgent*

In [ ]:
newAgentName = "AISearchWorkshopAgent"
input_prompt = "does perks plus cover gym membership?"

ex-2: Create new AI search agent

In [ ]:

conn_id = aisearch_connection.id
print(conn_id)

# Initialize agent AI search tool and add the search index connection id
ai_search = AzureAISearchTool(
    index_connection_id=conn_id, index_name=indexName, query_type=AzureAISearchQueryType.SEMANTIC, top_k=3, filter=""
)

# Create agent with AI search tool and process assistant run

agent = project_client.agents.create_agent(
    model=modelName,
    name=newAgentName,
    instructions="You are a helpful assistant",
    tools=ai_search.definitions,
    tool_resources=ai_search.resources
)

/subscriptions/74a008ce-fe34-4b7a-a4e7-012f6ca145c8/resourceGroups/aiworkshop2/providers/Microsoft.MachineLearningServices/workspaces/aiworkshop2-project1/connections/aiworkshop2aisearch


ex-3: Create thread and add messages to the thread

In [28]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
# Remember to update the message with your data
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content=input_prompt,
)
print(f"Created message, ID: {message.id}")

Created thread, ID: thread_xrl3OEcti0zlF5QI8LJHGdpX
Created message, ID: msg_c3TGddMp3lzfy4xN6pwmdeE3


ex-4: Run the agent and get the response

In [ ]:
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
print(f"Run finished with status: {run.status}")

if run.status == "failed":
    print(f"Run failed: {run.last_error}")

# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
print(f"Messages: {messages}")


Run finished with status: RunStatus.COMPLETED
Messages: {'object': 'list', 'data': [{'id': 'msg_X3z63wlpaIEhazaRBRHpDICm', 'object': 'thread.message', 'created_at': 1747408133, 'assistant_id': 'asst_rD1xfVotyUT44vhSemhl9KDm', 'thread_id': 'thread_xrl3OEcti0zlF5QI8LJHGdpX', 'run_id': 'run_THCoHGwM8v4y2cGdexBwG4cR', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': "Perks Plus may cover gym memberships depending on the specific benefits offered by your employer's plan. Many employer-sponsored wellness programs include gym membership discounts or reimbursements as part of their benefits. To get accurate information, you should check the details of your Perks Plus plan or contact your HR department for clarification on what is specifically covered.", 'annotations': []}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_c3TGddMp3lzfy4xN6pwmdeE3', 'object': 'thread.message', 'created_at': 1747408124, 'assistant_id': None, 'thread_id': 'thread_xrl3OEcti0zlF5QI8LJHGdpX', 'run

----

Response recvd: 

In [32]:
agent_response = messages.data[0].content[0].text.value# Get the content of the first message]
print(agent_response)

Perks Plus may cover gym memberships depending on the specific benefits offered by your employer's plan. Many employer-sponsored wellness programs include gym membership discounts or reimbursements as part of their benefits. To get accurate information, you should check the details of your Perks Plus plan or contact your HR department for clarification on what is specifically covered.


Clean up

In [ ]:
#Delete the assistant when done
project_client.agents.delete_agent(agent.id)
print("Deleted agent")
project_client.agents.delete_thread(thread.id)
print("Deleted thread")

ResourceNotFoundError: (None) No assistant found with id 'asst_rD1xfVotyUT44vhSemhl9KDm'.
Code: None
Message: No assistant found with id 'asst_rD1xfVotyUT44vhSemhl9KDm'.